In [1]:
import matplotlib.pyplot as plt
import json

%matplotlib inline

In [2]:
# Load main data

import os
from dataclasses import dataclass
import numpy as np


@dataclass
class Data:
    time: float
    psnr: float
    legal: bool
    stage: str
    ans: np.ndarray


levels = range(7)
qscales = [0.3, 0.4, 0.5, 0.6, 0.7]
timepoint = range(0, 600, 10)

base_dir = (
    "/home/xyhang/projects/VCIP2023-grand-challenge/experiments/SAv1/visualize/"
)

# Load bpp info

result_file = "results_visualize.json"

with open(result_file, "r") as f:
    result_data = json.load(f)

In [3]:
import matplotlib

In [4]:
filenames = [
    "IMG_1813",
    "IMG_3227",
    "IMG_3515",
    "IMG_3799",
    "IMG_4700",
    "IMG_6726",
    "IMG_8176",
]

def read_timeline_data(filename):
    jsonfile = os.path.join(base_dir, "PSNR/level-6/time_limit-1200/qscale-0.3", f"{filename}_statistics.json")
    with open(jsonfile, 'r') as f:
        data = json.load(f)
        results = []
        for step_results in data["step_results"]:
            results.append(
                Data(
                    psnr=-step_results["best_loss"],
                    legal=step_results["legal"],
                    time=step_results["time"],
                    stage=step_results["stage"],
                    ans=step_results["best_results"],
                )
            )
        return results

In [5]:
import site

site.addsitedir("/home/xyhang/projects/VCIP2023-grand-challenge/")

from tools.bdrate import BD_RATE

timestep = 10

bpps_time = [[] for k in range(len(filenames))] # 100 200 300 400 500 600
psnrs_time = [[] for k in range(len(filenames))]
islegal_time = [[] for k in range(len(filenames))]
ans_time = [[] for k in range(len(filenames))]

for i, filename in enumerate(filenames):
    bpp = result_data[filename]['bpp']
    results = read_timeline_data(filename)
    next_time = timestep
    for r in results:
        if r.time > next_time:
            bpps_time[i].append(bpp)
            psnrs_time[i].append(r.psnr)
            islegal_time[i].append(r.legal)
            ans_time[i].append(r.ans)
            next_time += timestep

In [6]:
from src.fileio import FileIO

qscale_idx = 0

fileio = FileIO(h=4480, w=6720, ctu_size=512, mosaic=False)

In [7]:
len(fileio.block_indexes)

126

In [8]:
def hex_to_bgr(hex_color):
    # Convert hex to RGB tuple
    hex_color = hex_color.lstrip("#")
    rgb = tuple(int(hex_color[i : i + 2], 16) for i in (0, 2, 4))
    # Convert RGB to BGR
    bgr = (rgb[2], rgb[1], rgb[0])
    return bgr

colors_hex = ["#D6D9D9", "#ADA091", "#6D1E0D", "#A83411", "#39150D", "#C4C78D"]
colors = [hex_to_bgr(color) for color in colors_hex]

In [9]:
colors

[(217, 217, 214),
 (145, 160, 173),
 (13, 30, 109),
 (17, 52, 168),
 (13, 21, 57),
 (141, 199, 196)]

In [10]:
import cv2

def generate_chosen_result_in_img(result):
    image = np.zeros((4480, 6720, 3), np.uint8)
    for i, bb in zip(result, fileio.block_indexes):
        cv2.rectangle(
            image, (bb[1], bb[0]), (bb[3], bb[2]), color=colors[i], thickness=cv2.FILLED
        )
        cv2.rectangle(
            image, (bb[1], bb[0]), (bb[3], bb[2]), color=(0, 0, 0), thickness=10
        )
    return image

In [11]:
for i, filename in enumerate(filenames):
    print(len(ans_time[i]))
    original_image = cv2.imread(
        f"/home/xyhang/projects/VCIP2023-grand-challenge/images/6720x4480/{filename}.png"
    )
    for d in range(4, len(ans_time[i]), 5):
        print(i, d)
        image_raw = generate_chosen_result_in_img(ans_time[i][d])
        image_merged = cv2.addWeighted(original_image, 0.3, image_raw, 0.7, 0)
        cv2.imwrite(f"output/{filename}_{d}.png", image_merged)

120


libpng warning: iCCP: known incorrect sRGB profile


0 4
0 9
0 14
0 19
0 24
0 29
0 34
0 39
0 44
0 49
0 54
0 59
0 64
0 69
0 74
0 79
0 84
0 89
0 94
0 99
0 104
0 109
0 114
0 119
120


libpng warning: iCCP: known incorrect sRGB profile


1 4
1 9
1 14
1 19
1 24
1 29
1 34
1 39
1 44
1 49
1 54
1 59
1 64
1 69
1 74
1 79
1 84
1 89
1 94
1 99
1 104
1 109
1 114
1 119
120


libpng warning: iCCP: known incorrect sRGB profile


2 4
2 9
2 14
2 19
2 24
2 29
2 34
2 39
2 44
2 49
2 54
2 59
2 64
2 69
2 74
2 79
2 84
2 89
2 94
2 99
2 104
2 109
2 114
2 119
120


libpng warning: iCCP: known incorrect sRGB profile


3 4
3 9
3 14
3 19
3 24
3 29
3 34
3 39
3 44
3 49
3 54
3 59
3 64
3 69
